In [1]:
def runNN(train_features, train_label, test_features, test_label, n_neurons, n_epochs, seed=155,
          data_dim = 13, timesteps = 1, batch_size = 52, states = temp_data.keys(),
          history=True, del_files=True, early_stopping=None):
    np.random.seed(seed)
    #create model
    rnn_model = simple_rnn_model(data_dim,timesteps,batch_size,states, n_neurons)
    model_callbacks = []
    if early_stopping is not None:
        model_callbacks = [early_stopping]
    if history:
        processed_dir = os.path.join(os.getcwd(),os.pardir, 'models/')
        filepath= processed_dir + "nn_weights_%dneurons-{epoch:02d}.hdf5" %n_neurons
        checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_weights_only=True,
                                     save_best_only=False, mode='max')
        model_callbacks.append(checkpoint)
        print("model checkpoint")
        output = rnn_model.fit(train_features,
                               train_label[0], #label for the targeted state
                               validation_data= (
                                   test_features,
                                   test_label[0]),
                               epochs=n_epochs, 
                               verbose=0, 
                               batch_size=batch_size,
                               callbacks=model_callbacks, 
                               initial_epoch=0).history
        
        print("print loss")
    
        time.sleep(1) # hack so that files can be opened in subsequent code
        #create model
        temp_val_model = simple_rnn_model(data_dim,timesteps,batch_size,states, n_neurons)
        test_over_time = []
        
        for i in range(len(output['loss'])):
            temp_val_model.load_weights(processed_dir + "nn_weights_%dneurons-%02d.hdf5" % (n_neurons,i))
            scores = temp_val_model.evaluate(X_test_set, Y_test_set, verbose=0)
            test_over_time.append(scores)
            # delete files once we're done with them
            if del_files:
                os.remove(processed_dir + "nn_weights_%dneurons-%02d.hdf5" % (n_neurons,i))
        
        test_over_time = np.array(test_over_time)
        output['test_loss'] = [row[0] for row in test_over_time]
        output['test_acc'] = [row[1] for row in test_over_time]
    else:
        model_output = rnn_model.fit(train_features,
                               train_label[0], #label for the targeted state
                               validation_data= (
                                   test_features,
                                   test_label[0]),
                               epochs=n_epochs, 
                               verbose=0, 
                               batch_size=batch_size,
                               callbacks=model_callbacks, 
                               initial_epoch=0).history
        validation_size = 0
        output = {}
        training_size = train_features[0].shape[0] - validation_size
        train_scores = rnn_model.evaluate(train_features, train_label[0], verbose=0)
        test_scores = rnn_model.evaluate(test_features, test_label[0], verbose=0)
        output['train_loss'] = train_scores[0]
        output['train_acc'] = train_scores[1]
        output['test_loss'] = test_scores[0]
        output['test_acc'] = test_scores[1]
        
    return output

NameError: name 'temp_data' is not defined

In [ ]:
my_second_nn = runNN(train_features, train_label, test_features, test_label, 50, 100)
fig, (ax1, ax2) = plt.subplots(2, 1)
ax1.plot(range(len(my_second_nn['loss'])),my_second_nn['loss'], color='blue',label='Training Set', lw=1.5)
ax1.plot(range(len(my_second_nn['test_loss'])),my_second_nn['test_loss'], color='green',label='Test Set', lw=1.5)
ax2.plot(range(len(my_second_nn['acc'])),my_second_nn['acc'], color='blue',label='Training Set', lw=1.5)
ax2.plot(range(len(my_second_nn['test_acc'])),my_second_nn['test_acc'], color='green',label='Test Set', lw=1.5)
leg = ax1.legend(bbox_to_anchor=(0.7, 0.9), loc=2, borderaxespad=0.,fontsize=13)
ax1.set_xticklabels('')
ax2.set_xlabel('# Epochs',fontsize=14)
ax1.set_ylabel('Loss',fontsize=14)
ax2.set_ylabel('Accuracy',fontsize=14)
plt.tight_layout()
plt.show()
end = time.time()